In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [ ]:
!pip install wandb datasets transformers sacrebleu==1.5.1  bert-score -qqq

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 287 kB 45.5 MB/s 
     |████████████████████████████████| 2.9 MB 31.9 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 59 kB 5.3 MB/s 
     |████████████████████████████████| 180 kB 44.8 MB/s 
     |████████████████████████████████| 139 kB 45.4 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 243 kB 49.5 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 125 kB 46.4 MB/s 
     |████████████████████████████████| 1.3 MB 37.9 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 271 kB 43.1 MB/s 
     |███████████████████

In [ ]:
!nvidia-smi

Fri Oct 15 15:10:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import datetime
from tqdm import tqdm
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d")
print(todaydate)
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'sparql-qald9-translator-{todaydate}-{model_checkpoint}' + '-fp16' if fp16 else ''
model_path='./models/'+model_name
ds_path= '../../../data/dataset/qald-text-to-sparql'


2021-10-15


In [ ]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', name= f'{todaydate}-{model_checkpoint}-QALD')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
model_name

'sparql-qald9-translator-2021-10-15-t5-small-fp16'

In [ ]:
!mkdir models

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, T5TokenizerFast

prefix = "translate english to sparql: "
model_name = "yazdipour/text-to-sparql-t5-small-2021-10-15_01-00"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = T5TokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!apt install git-lfs -qqq
!git config --global credential.helper store

In [ ]:
!unzip qald-text-to-sparql.zip

Archive:  qald-text-to-sparql.zip
 extracting: qald-text-to-sparql/dataset_dict.json  
  inflating: qald-text-to-sparql/test.csv  
   creating: qald-text-to-sparql/test/
  inflating: qald-text-to-sparql/test/dataset.arrow  
  inflating: qald-text-to-sparql/test/dataset_info.json  
  inflating: qald-text-to-sparql/test/state.json  
  inflating: qald-text-to-sparql/train.csv  
   creating: qald-text-to-sparql/train/
  inflating: qald-text-to-sparql/train/dataset.arrow  
  inflating: qald-text-to-sparql/train/dataset_info.json  
  inflating: qald-text-to-sparql/train/state.json  


In [ ]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk('qald-text-to-sparql')
# raw_datasets = load_dataset("yazdipour/text-to-sparql-t5-lc-quad-v2", data_files={"train": "train.csv", "test": "test.csv"})

In [ ]:
# Preprocessing

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
print(raw_datasets['test']['translation'][0]['sparql'],'\n',
      raw_datasets['test']['translation'][0]['en'],'\n',
      tokenizer(raw_datasets['test']['translation'][0]['en']))

select distinct ?uri where [ res:Salt_Lake_City onto:timeZone ?uri ] 
 what is the time zone of salt lake city? 
 {'input_ids': [125, 19, 8, 97, 2901, 13, 3136, 6957, 690, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry
print('\n',max_input_length, max_target_length)

100%|██████████| 408/408 [00:00<00:00, 150759.94it/s]


 85 299


In [ ]:
source_lang = "en"
target_lang = "sparql"
prefix = "translate english to sparql: "

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 150
    })
})

In [ ]:
# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=fp16,
    push_to_hub=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]
    return preds, labels

In [ ]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    P, R, F1 = scorer.score(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    gen_len = np.mean(prediction_lens)
    
    return {"gen_len":gen_len, 'P':P.mean(), 'R':R.mean(), 'F1':F1.mean(), "bleu-score": result["score"], "bleu-precisions": result["precisions"], "bleu-bp": result["bp"]}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-15_01-00 into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.0k/231M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

Download file runs/Oct15_01-02-30_244be396bb35/1634260154.6301315/events.out.tfevents.1634260154.244be396bb35.…

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Clean file runs/Oct15_01-02-30_244be396bb35/1634260154.6301315/events.out.tfevents.1634260154.244be396bb35.83.…

Download file runs/Oct15_01-02-30_244be396bb35/events.out.tfevents.1634260154.244be396bb35.83.0:  56%|#####5  …

Clean file runs/Oct15_01-02-30_244be396bb35/events.out.tfevents.1634260154.244be396bb35.83.0:  16%|#5        |…

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

Using amp fp16 backend


In [ ]:
import torch

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 408
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 26
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Gen Len,P,R,F1,Bleu-score,Bleu-precisions,Bleu-bp
1,No log,4.148813,19.000000,0.236769,-0.030384,0.100305,0.886776,"[56.84848484848485, 25.0, 8.88888888888889, 0.041666666666666664]",0.185136


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 16
Trainer is attempting to log a value of "[56.84848484848485, 25.0, 8.88888888888889, 0.041666666666666664]" of type <class 'list'> for key "eval/bleu-precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=26, training_loss=4.095533517690805, metrics={'train_runtime': 31.9953, 'train_samples_per_second': 12.752, 'train_steps_per_second': 0.813, 'total_flos': 13145072467968.0, 'train_loss': 4.095533517690805, 'epoch': 1.0})

In [ ]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/sparql-qald9-translator-2021-10-15-t5-small-fp16
Configuration saved in ./models/sparql-qald9-translator-2021-10-15-t5-small-fp16/config.json
Model weights saved in ./models/sparql-qald9-translator-2021-10-15-t5-small-fp16/pytorch_model.bin
tokenizer config file saved in ./models/sparql-qald9-translator-2021-10-15-t5-small-fp16/tokenizer_config.json
Special tokens file saved in ./models/sparql-qald9-translator-2021-10-15-t5-small-fp16/special_tokens_map.json


In [ ]:
model_name

'yazdipour/text-to-sparql-t5-small-2021-10-15_01-00'

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to yazdipour/text-to-sparql-t5-small-2021-10-15_01-00
Configuration saved in yazdipour/text-to-sparql-t5-small-2021-10-15_01-00/config.json
Model weights saved in yazdipour/text-to-sparql-t5-small-2021-10-15_01-00/pytorch_model.bin
tokenizer config file saved in yazdipour/text-to-sparql-t5-small-2021-10-15_01-00/tokenizer_config.json
Special tokens file saved in yazdipour/text-to-sparql-t5-small-2021-10-15_01-00/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.36k/231M [00:00<?, ?B/s]

Upload file runs/Oct15_15-12-52_2a81451c8faa/events.out.tfevents.1634310934.2a81451c8faa.76.0:  68%|######8   …

Upload file runs/Oct15_15-12-52_2a81451c8faa/1634310934.6802654/events.out.tfevents.1634310934.2a81451c8faa.76…

Upload file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

To https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-15_01-00
   b1831eb..27808f4  main -> main

Dropping the following result as it does not have all the necessary field:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-15_01-00
   27808f4..371c071  main -> main



'https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-15_01-00/commit/27808f4dd2620ec39fbfc42511efe13a1d365066'